### Full algorithm
HCHS_data(cleaning) -> replace abbrviations -> translation with Hugging Face -> rid of unnecessory information

#### HCHS data

In [35]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
from transformers import pipeline

In [17]:
HCHS_data_second=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/VM_ _HCHS_10000_20210708.xlsx')
HCHS_data_second['Langname']=HCHS_data_second['Langname'].replace([' ', ''], np.nan)
HCHS_Langname_second=HCHS_data_second['Langname'].str.strip()
HCHS_Langname_second=HCHS_Langname_second.dropna(axis=0)

In [18]:
HCHS_data=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/VM_Soarian_HCHS_20210422.xlsx')
HCHS_data['Langname']=HCHS_data['Langname'].replace([' ', ''], np.nan)
HCHS_Langname=HCHS_data['Langname'].str.strip()
HCHS_Langname=HCHS_Langname.dropna(axis=0)

In [19]:
HCHS_Langname= HCHS_Langname.append(HCHS_Langname_second, ignore_index = True)

/tmp/ipykernel_395/81917566.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HCHS_Langname= HCHS_Langname.append(HCHS_Langname_second, ignore_index = True)


In [20]:
HCHS_Langname

0       Liegt bei Ihnen eine vom Arzt diagnostizierte ...
1       Liegt bei Ihnen ein vom Arzt diagnostizierter ...
2                          Wann war die letzte Injektion?
3                          Wann war die letzte Injektion?
4       Hatten Sie bereits einen vom Arzt diagnostizie...
                              ...                        
2669                                Sonstiges |  nämlich:
2670    Ich habe das Gefühl |  dass ich über Sexualitä...
2671           Ich bin motiviert |  sexuell aktiv zu sein
2672    Sexualität ist in meinem Leben eher durch Umst...
2673    In Bezug auf sexuelle Kontakte bin ich ängstli...
Name: Langname, Length: 2674, dtype: object

In [21]:
df=pd.DataFrame()
df['HCHS_German']=HCHS_Langname

In [22]:
df['HCHS_German']=df['HCHS_German'].str.replace(r'[{4}|]', ',', regex=True)#cases with |

#### Abbriviation replacement

In [23]:
list_abbr=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/german_abbreviation.xlsx')

In [24]:
df['HCHS_German_without_abbreviation']=df['HCHS_German']

In [25]:
#abbreviation replacment
for i in tqdm(range(list_abbr.shape[0])):
    abbreviation=list_abbr['Abbreviation'][i]
    description=list_abbr['Description'][i]
    df['HCHS_German_without_abbreviation']=df['HCHS_German_without_abbreviation'].str.replace(r'\b' + abbreviation + r'[^\w]' ,description+' ',  regex= True)

100%|██████████| 874/874 [00:17<00:00, 48.89it/s]


#### Translation German-English

In [26]:
model="Helsinki-NLP/opus-mt-de-en"
translator=pipeline('translation', model)

In [27]:
tr=translator(df['HCHS_German_without_abbreviation'].to_list())

In [28]:
translation_list=[]
for i in tr:
    translation_list.append(i.get('translation_text'))

In [29]:
df['HCHS_English']=translation_list

#### Rid of unnecessory information

In [36]:
nlp=spacy.load('en_core_web_lg')

In [41]:
df=pd.read_csv('Source_to_concept_map.csv',  on_bad_lines='skip', delimiter=';')

In [42]:
df

,source_code,source_concept_id,source_vocabulary_id,source_code_description,target_concept_id,target_vocabulary_id,valid_start_date,valid_end_date,invalid_reason
0,Saliva weight,0,HCHS - Varaibles,Saliva,4120374,SNOMED,31.01.2002,31.12.2099,NaN
1,Non-ischemic heart disease,0,HCHS - Varaibles,Non-ischemic heart disease,321588,SNOMED,31.01.2002,31.12.2099,NaN
2,Watching television,0,HCHS - Varaibles,Watching television,45757640,SNOMED,31.01.2015,31.12.2099,NaN
3,Ischemic heart disease,0,HCHS - Varaibles,Ischemic heart disease,4185932,SNOMED,31.01.2005,31.12.2099,NaN
4,Valvular heart disease,0,HCHS - Varaibles,Valvular heart disease,4281749,SNOMED,31.01.2002,31.12.2099,NaN
...,...,...,...,...,...,...,...,...,...
84,"Pain , Physical discomfort",0,HCHS - Varaibles,"Pain , Physical discomfort",37398847,SNOMED,01.04.2016,31.12.2099,NaN
85,How much was your pain last week?,0,HCHS - Varaibles,How much was your pain last week?,4234651,SNOMED,31.01.2004,31.12.2099,NaN
86,Back pain,0,HCHS - Varaibles,Back pain,134736,SNOMED,31.01.2006,31.12.2099,NaN
87,Headache,0,HCHS - Varaibles,Headache,378253,SNOMED,31.01.2002,31.12.2099,NaN


In [43]:
english_ready=[]
for sentence in df['source_code'].to_list():
    token_list=[]
    doc=nlp(sentence)
    token_list=[token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.text!='doctor']
    text = " ".join(token_list)
    english_ready.append(text)

In [31]:
english_ready=[]
for sentence in df['HCHS_English'].to_list():
    token_list=[]
    doc=nlp(sentence)
    token_list=[token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.text!='doctor']
    text = " ".join(token_list)
    english_ready.append(text)

In [44]:
df['prepeared_source_code']=english_ready

In [32]:
df['HCHS_new']=english_ready

In [45]:
df

,source_code,source_concept_id,source_vocabulary_id,source_code_description,target_concept_id,target_vocabulary_id,valid_start_date,valid_end_date,invalid_reason,prepeared_source_code
0,Saliva weight,0,HCHS - Varaibles,Saliva,4120374,SNOMED,31.01.2002,31.12.2099,NaN,saliva weight
1,Non-ischemic heart disease,0,HCHS - Varaibles,Non-ischemic heart disease,321588,SNOMED,31.01.2002,31.12.2099,NaN,non ischemic heart disease
2,Watching television,0,HCHS - Varaibles,Watching television,45757640,SNOMED,31.01.2015,31.12.2099,NaN,watch television
3,Ischemic heart disease,0,HCHS - Varaibles,Ischemic heart disease,4185932,SNOMED,31.01.2005,31.12.2099,NaN,ischemic heart disease
4,Valvular heart disease,0,HCHS - Varaibles,Valvular heart disease,4281749,SNOMED,31.01.2002,31.12.2099,NaN,Valvular heart disease
...,...,...,...,...,...,...,...,...,...,...
84,"Pain , Physical discomfort",0,HCHS - Varaibles,"Pain , Physical discomfort",37398847,SNOMED,01.04.2016,31.12.2099,NaN,pain physical discomfort
85,How much was your pain last week?,0,HCHS - Varaibles,How much was your pain last week?,4234651,SNOMED,31.01.2004,31.12.2099,NaN,pain week
86,Back pain,0,HCHS - Varaibles,Back pain,134736,SNOMED,31.01.2006,31.12.2099,NaN,pain
87,Headache,0,HCHS - Varaibles,Headache,378253,SNOMED,31.01.2002,31.12.2099,NaN,headache


In [46]:
df.to_excel('source_to_concept_map_new.xlsx', encoding='utf-16')

In [34]:
df.to_excel('HCHS_full_cycle_plus.xlsx', encoding='utf-16')